# Decorators

In Python decorators are a software pattern used to extend the functionality to an object (a function or class) at compile time. In practice the basic decorator is a function that takes a function and returns the modified version of that function. Decorators helps in several ways, for example it helps programmers to avoid code repetitions, mantain backward compatibility and simply add or remove functionalities of objects.

## First example

Consider the case in which we have several functions of which we want to estimate the execution time. We don't want to modify each function to include a "timer" but we can obtain the same result by adding a decorator to each of those function, so we just need to program one decorator and apply it where we need. 

Consider the following decorator.

In [81]:
import time
def CalcTime(input_func):  # this is the decorator
    def decorated_func(*args):  # here we extend the functionality of "input_func"
        time1 = time.perf_counter()
        input_func(*args)
        print("time = {:1.6f} seconds".format(time.perf_counter() - time1))
    return decorated_func 

Notice that we can pass arguments to the function that we decorate, in this case I pass a generic `*args`. Moreover the decorator has to return the decorated function. Now we just need to apply the decorator to a function. For example:

In [82]:
import numpy as np

@CalcTime
def long_calc(iterations):
    for i in range(1, iterations):
        np.log(i)

In [83]:
long_calc(10000)

time = 0.0175664000 seconds


Not all timing functions in the `time` module have a good resolution, and it depend on the OS. Here's a quick breakdown of the main timers in the `time` module on windows: 

In [84]:
for timer in ['monotonic', 'perf_counter', 'process_time', 'time']:
    print(time.get_clock_info(timer))

namespace(adjustable=False, implementation='GetTickCount64()', monotonic=True, resolution=0.015625)
namespace(adjustable=False, implementation='QueryPerformanceCounter()', monotonic=True, resolution=1e-07)
namespace(adjustable=False, implementation='GetProcessTimes()', monotonic=True, resolution=1e-07)
namespace(adjustable=True, implementation='GetSystemTimeAsFileTime()', monotonic=False, resolution=0.015625)


You see why I used the `perf_counter`. Let's go back to decorators. If we had another function of which we want to measure its performances we can just add the decorator:

In [85]:
@CalcTime
def long_calc_2(iterations):
    for i in range(iterations):
        np.sqrt(i)

In [86]:
long_calc_2(10000)

time = 0.0180103000 seconds


## Decoators with arguments

Even if we can apply the same decorators to many functions, shortening the code and avoiding repetitions, we may be in the situation in which we need many decorators with a slightly different behavior when applied to slightly different functions. For example, what if we wanted to measure the run time of functios just like before, but we also want to add a specific functionality (in our case simply modifying the printed string) depending on which calculation (log or sqrt) we are performing. We can then use a decorator with arguments:

In [87]:
def DecFun(operation):  # this is a function that returns a decorator
    def CalcTime(input_func):  # decorator
        def decorated_func(*args): 
            time1 = time.perf_counter()
            input_func(*args)
            print("time of {} function = {:1.6f} seconds".format(operation, time.perf_counter() - time1))
        return decorated_func
    return CalcTime

In [88]:
@DecFun('logarithm')
def long_calc(iterations):
    for i in range(1, iterations):
        np.log(i)

In [89]:
long_calc(10000)

time of logarithm function = 0.0255265000 seconds


In [90]:
@DecFun('square root')
def long_calc_2(iterations):
    for i in range(iterations):
        np.sqrt(i)

In [91]:
long_calc_2(10000)

time of square root function = 0.0245481000 seconds


## Classes as Decorators

At this point we can extend the functionality of functions or class methods. What about *decorating functions using classes*? It is indeed possible. Our basic recipe is to write a function that takes a parameter. In this case instead we write a `class` that returns the input function with the modified behavior.

In [371]:
class ClassDecorator(object):    # decorates the class
    def __init__(self, func):
        self.func = func
    def __call__(self, *args):
        print("The function '{}' has been called with arguments: {}".format(self.func, *args))

In [372]:
@ClassDecorator
def long_calc(iterations):
    for i in range(1, iterations):
        np.log(i)

In [373]:
long_calc(10000)

The function '<function long_calc at 0x0000021AFA568730>' has been called with arguments: 10000


Why implementing `__call__`? the `@ClassDecorator` applied to `long_calc` creates the new version of `long_calc` as described in `ClassDecorator` and calls `ClassDecorator`itself therfore the `__call__`is automatically invoked.